In [1]:
import sys
import os
import warnings

# Ajouter le répertoire parent pour inclure 'src'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

# Imports après avoir ajusté le sys.path
from src.backtester.back_tester import BackTesting
from src.strategies.strategies import *
from src.utils.utilities import Utilities
from src.performance.graph import IndexPlotter
import utils.constant as constant
from src.performance.metrics import MetricsCalculator
from datetime import datetime, date

# Supprimer les avertissements FutureWarning et RuntimeWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

pd.set_option('display.float_format', '{:.3f}'.format)

<div style="background-color: rgb(15, 50, 110); color: white; padding: 15px; border-radius: 8px; box-shadow: 0 3px 6px 0 rgba(0, 0, 0, 0.2); text-align: center;">
    <h1 style="font-size: 32px; font-weight: bold; margin: 0;">Volatility Timing under Low-Volatility Strategy</h1>
</div> 

## <span style="color: rgb(15, 50, 150); font-size: 20px; font-weight: bold;"> Volatility Strategies </span>

In [2]:
# Data import
asset_indices_monthly = Utilities.load_asset_indices(["LowVolatilityDecile", "VolatilityTiming", 
                                                      "HighVolatilityDecile"], 
                                                    folder_subpath="asset_indices\\monthly_eq_weighted")

asset_indices_monthly_all = Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile", 
                                                          "HighVolatilityDecile", "VolatilityTiming", 
                                                          "VolatilityTiming2sided"], 
                                                          folder_subpath="asset_indices\\monthly_eq_weighted")

other_data = Utilities.get_data_from_pickle('other_US_data', folder_subpath="universe")

# Parameters
risk_free_rate_ticker = "US0003M Index"
ticker = "RIY INDEX"
start_date = datetime(2000, 1, 31)
end_date = datetime(2024, 2, 29)
frequency = "monthly"


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\naiml\\Desktop\\M272\\Bloomberg\\Python-API-Bloomberg\\data\\universe\\other_US_data.pkl'

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;"> 1. Volatility and Return Profile under Different Market Conditions </span>

In [ ]:
# Filtering asset indices
asset_indices = {ticker :item for ticker, item in asset_indices_monthly_all.items() if ticker in 
                 ["LowVolatilityDecile", "MidVolatilityDecile", "HighVolatilityDecile"]}

# Plot asset indices under different conditions
IndexPlotter.asset_indices_plot_under_diff_conditions(asset_indices, other_data, risk_free_rate_ticker,
                                                      start_date, end_date,
                                                      frequency, constant.REBALANCING_MOMENT, ticker)

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;"> 2. Performance and Wealth Plots of the Volatility Strategies </span>

In [ ]:
# Graph des tracks records Vol Timing
IndexPlotter.plot_track_records(asset_indices_monthly, other_data['USRINDEX Index'])

In [ ]:
# Convertir toutes les dates dans other_data en datetime avec UTC
for key in other_data:
    if isinstance(other_data[key].index[0], (pd.Timestamp, datetime)):
        other_data[key].index = pd.to_datetime(other_data[key].index, utc=True)

# Vérifier la conversion des dates
for key in other_data:
    print(f"Key: {key}, Date index type: {type(other_data[key].index[0])}")

# Calcul de la performance des switchs
metrics_calculator = MetricsCalculator(other_data, risk_free_rate_ticker)
vol_timing_strategy = VolatilityTimingStrategy(start_date, frequency, constant.REBALANCING_MOMENT, "Equally Weighted")
switch_performance = metrics_calculator.calculate_switch_performance(vol_timing_strategy, other_data, list(other_data.keys()), start_date, end_date)

# Afficher la performance des switchs
switch_performance_df = pd.DataFrame(switch_performance, index=[0]).T
print(switch_performance_df)


In [ ]:
IndexPlotter.asset_indices_barplot(asset_indices_monthly_all, other_data, risk_free_rate_ticker)

In [ ]:
# Graph des tracks records des stratégies
asset_indices = {ticker :item for ticker, item in asset_indices_monthly_all.items() if ticker in 
                 ["LowVolatilityDecile", "MidVolatilityDecile", "HighVolatilityDecile", "VolatilityTiming2sided"]}

IndexPlotter.plot_track_records(asset_indices, other_data['USRINDEX Index'])

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  4. Performance and Risk Measures of the Various Volatility Portfolios </span>

In [ ]:
perf_metrics_df = IndexPlotter.display_asset_indices_metrics(asset_indices_monthly_all, 
                                                             other_data,risk_free_rate_ticker)
perf_metrics_df

## <span style="color: rgb(15, 50, 150); font-size: 20px; font-weight: bold;"> Robustness Tests </span>


### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  1. CAPM and Fama–French Alphas </span>

In [ ]:
CPAM_stats, FF_stats = IndexPlotter.display_regress_statistics(asset_indices_monthly_all)
CPAM_stats

In [ ]:
FF_stats

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  2. Performance plots for various rebalancing frequencies </span>

In [ ]:
# Chargement des indices
asset_indices_quaterly = Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile", "HighVolatilityDecile", 
                                              "VolatilityTiming", "VolatilityTiming2sided"],
                                             folder_subpath="asset_indices\\quarterly_eq_weighted")
asset_indices_semiannually = Utilities.load_asset_indices(["VolatilityTiming"],
                                             folder_subpath="asset_indices\\semiannually_eq_weighted")
asset_indices_annually = Utilities.load_asset_indices(["VolatilityTiming"],
                                             folder_subpath="asset_indices\\annually_eq_weighted")

In [ ]:
# Graph des tracks records des stratégies : rebalancement trimestriel (fin trimestre)
IndexPlotter.plot_track_records(asset_indices_quaterly, other_data['USRINDEX Index'])

In [ ]:
# Graphique des tracks records de la stratégie VolTiming pour différentes fréquences de rebalancement
IndexPlotter.plot_tracks_general( [asset_indices_monthly['VolatilityTiming'],
                                    asset_indices_quaterly['VolatilityTiming'],
                                    asset_indices_semiannually['VolatilityTiming'],
                                    asset_indices_annually['VolatilityTiming']],
                                    label_names=["monthly", "quarterly","semiannually","annually"],
                                    graph_title="VolatilityTiming - Wealth plot under different rebalancing frequencies")

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  3. Performance under transaction costs </span>

In [ ]:
asset_indices_with_tc = Utilities.load_asset_indices(["LowVolatilityDecile", "HighVolatilityDecile", "VolatilityTiming"],
                                             folder_subpath="asset_indices\\monthly_with_transaction_costs")

asset_indices_with_tc_all = Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile","HighVolatilityDecile",
                                                           "VolatilityTiming","VolatilityTiming2sided"],
                                             folder_subpath="asset_indices\\monthly_with_transaction_costs")

IndexPlotter.plot_track_records(asset_indices_with_tc, other_data['USRINDEX Index'])

In [ ]:
perf_metrics_with_tc_df = IndexPlotter.display_asset_indices_metrics(asset_indices_with_tc_all, 
                                                                     other_data, risk_free_rate_ticker)

IndexPlotter.display_joint_metrics(perf_metrics_df, perf_metrics_with_tc_df, 
                                   label_names=['High Volatility','Low Volatility','Volatility Timing'],
                                   column_names=["Without TC", "With TC"])

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  4. Performance under different weighting choices </span>

In [ ]:
# Data import
asset_indices_vol_scaling= Utilities.load_asset_indices(["LowVolatilityDecile", "HighVolatilityDecile", "VolatilityTiming"],
                                             folder_subpath="asset_indices\\monthly_vol_scaling")

asset_indices_vol_scaling_all= Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile","HighVolatilityDecile",
                                                           "VolatilityTiming","VolatilityTiming2sided"],
                                             folder_subpath="asset_indices\\monthly_vol_scaling")

asset_indices_max_div = Utilities.load_asset_indices(["LowVolatilityDecile", "HighVolatilityDecile", "VolatilityTiming"],
                                             folder_subpath="asset_indices\\monthly_max_div")


asset_indices_max_div_all = Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile","HighVolatilityDecile",
                                                           "VolatilityTiming","VolatilityTiming2sided"],
                                             folder_subpath="asset_indices\\monthly_max_div")

In [ ]:
# Graphique des tracks records de la stratégie VolTiming pour différentes méthodes de calcul de poids
IndexPlotter.plot_tracks_general( [asset_indices_monthly_all['VolatilityTiming'],
                                    asset_indices_vol_scaling['VolatilityTiming'],
                                    asset_indices_max_div['VolatilityTiming']],
                                    label_names=["equally weigthed", "volatility scaling","max diversification"],
                                    graph_title="VolatilityTiming - Wealth plot under different weighting choices")

In [ ]:
perf_metrics_vol_scaling_df = IndexPlotter.display_asset_indices_metrics(asset_indices_vol_scaling_all, 
                                                                     other_data, risk_free_rate_ticker)

perf_metrics_max_div_df = IndexPlotter.display_asset_indices_metrics(asset_indices_max_div_all, 
                                                                     other_data, risk_free_rate_ticker)

In [ ]:
IndexPlotter.display_joint_metrics(perf_metrics_df, perf_metrics_vol_scaling_df,  perf_metrics_max_div_df,
                                   label_names=['High Volatility','Low Volatility','Volatility Timing'],
                                   column_names=["equally weigthed", "volatility scaling","max diversification"])